# Connected Components

The purpose of this assignment is to familiarize yourself with the handling of graph data structures. You will implement the algorithm for identifying the connected components of an undirected graph, implementing depth-first-search as a subroutine along the way.

You will use the [NetworkX](https://networkx.github.io/) Python package to represent and manipulate graphs. You should first familiarize yourself with its functionality by going through the brief [tutorial](http://networkx.github.io/documentation/networkx-1.9.1/tutorial/index.html). For this homework, you may only use the basic undirected graph methods listed [here](http://networkx.github.io/documentation/networkx-1.9.1/reference/classes.graph.html).

As a use case, we will work with a dataset recording the interactions between characters in Homer's *Iliad*.

In [22]:
import urllib2
homer = urllib2.urlopen('http://people.sc.fsu.edu/~jburkardt/datasets/sgb/homer.dat')

The format of the data is straightforward. After some comment lines (beginning with \*), the file lists a codename for each character (i.e., node of the graph), followed by a description. The file then lists the groups of characters that interact in each chapter, from which you will form the edges. For instance, the first line has the form:

```1:CH,AG,ME,GS;AP,CH;HE,AC;AC,AG,CA;HE,AT;AT,AC;AT,OG;NE,AG,AC;CS,OD```

This means that CH,AG,ME,GS interacted, so there are edges for all pairs of these nodes. Groups of characters that interacted are separated by semicolons. The lines start with chapter information of the form `1:` or `&:`, which can be ignored for this problem.

First implement a function to read in the nodes from the input file. You may implement any auxiliary functions as needed, and are encouraged to use small functions with specific purposes to keep your code readable. Any function you implement should be clearly commented.

In [23]:
def read_nodes(gfile):
    lst1=[]                              #create list1, which will store nodes
    while True:
        line = gfile.readline().strip()  # read the data in gfile line by line
        if line == "":     
            break                        # if the line is blank, then jump out of the loop
        lst1.append(line.split(" ")[0])  # add the first element of every line into the list1  
    while "*" in lst1:                   # find and remove all"*" in list1
         lst1.remove("*")
    return lst1

Next implement a function to read in the edges from the input file.

In [24]:
def read_edges(gfile):
    lst2=[]
    lst3=[]                               # create two lists
    while True:
        line = gfile.readline().strip()   # continue read the data from in gfile line by line
        if "End" in line :                # if the line has "End", then jump out of the loop
            break
        for i in line.strip().split(':')[1].split(';'): # remove "1:"and "&:" and split every line with ";"
            lst2.append(i.split(','))                   # split every i with"," and then add into list2
        for a in range(0,len(lst2)):                
            for b in range(0,len(lst2[a])):         
                for c in range(b+1,len(lst2[a])):       # create every pair for the elements in ";"
                    lst3.append((lst2[a][b],lst2[a][c]))# and add them into list3   
    return lst3

The following code should now correctly create the graph.

In [25]:
import networkx as nx
G = nx.Graph()
G.add_nodes_from(read_nodes(homer))
G.add_edges_from(read_edges(homer))

Next implement depth-first search (DFS). The function takes in a graph and a root node, and returns a list of the nodes visited during the search. The nodes should appear in the order in which they were *first visited*. The neighbors of a node should be processed in *alphabetical order*, where numbers come before letters. This will ensure that the output of your function is uniquely defined, given any input node.

In [26]:
def DFS(graph,root):
    visited=[]
    def search(u):                     # define a subfunction search
        a=[]
        a=sorted(graph.neighbors(u))   # sort the neighbors of node u
        for i in a:
            if i not in visited:
                visited.append(i)
                search(i)              # if a neighbor isn't visited, then add into visited and search this node
    if root not in visited:            # so we start from the first node, which is root
        visited.append(root)
        search(root)    
    return visited   

We will check the correctness of your code by verifying that it correctly computes the DFS tree starting at Ulysses (node `OD`).

In [27]:
ulysses = DFS(G, 'OD')

Next implement a function that finds the connected components of the character graph. When choosing roots for your components, always pick the *smallest unvisited node* according to alphabetical ordering. Combined with your DFS routine, this will ensure that the output is again uniquely defined.

In [28]:
def connected_components(graph):
    existed=[]                   # create a list called existed to represent what we've already discovered
    a=sorted(graph.nodes())      # sort all nodes in the graph
    group=[DFS(G,a[0])]          # put the result of DFS in the first node in a list"group"
    existed.extend(group[0])     # add the elements in group into existed
    for i in a:
        if i not in existed:
            group.extend([DFS(G,i)]) # if find new node which doesn't belong to existed, add its dfs result to group
            existed.extend(DFS(G,i)) # then add dfs result in existed 
    return group

We will check correctness of your code by verifying that your output list is identical to our solution.

In [29]:
character_interactions = connected_components(G)

As a preliminary check, you should find that the following statements are all true.

In [30]:
component_sizes = [len(c) for c in character_interactions]
print "There are 12 connected components in the Iliad:", len(component_sizes) == 12
print "The giant component has size 542:", max(component_sizes) == 542
print "There are 5 isolated characters:", len([c for c in component_sizes if c == 1]) == 5

There are 12 connected components in the Iliad: True
The giant component has size 542: True
There are 5 isolated characters: True
